# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from dataset_manager import MovieLensManager, ROOT
from llm_manager import VanillaBertClassifier

In [2]:
EPOCHS = 8
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

In [4]:
kg_manager.llm_df[kg_manager.llm_df["split"] == "test"]["labels"].unique()

array([1, 0], dtype=int64)

llm_df = kg_manager.llm_df.merge(kg_manager.target_df[["id", "prompt_feature_title", "prompt_feature_genres"]].rename(columns={"id": "target_id"}), on = "target_id")
llm_df

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [5]:
kg_manager.data

HeteroData(
  source={ node_id=[610] },
  target={
    node_id=[9742],
    x=[9742, 20],
  },
  (source, edge, target)={ edge_index=[2, 100836] },
  (target, rev_edge, source)={ edge_index=[2, 100836] }
)

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [6]:
VANILLA_ROOT = f"{ROOT}/llm/vanilla"

In [7]:
vanilla_bert_classifier = VanillaBertClassifier(
    kg_manager.llm_df,
    kg_manager.source_df,
    kg_manager.target_df,
    root_path=VANILLA_ROOT,
)

Some weights of BertForSequenceClassificationRanges were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassificationRanges were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized because the shapes did not match:
- bert.embeddings.token_type_embeddings.weight: found shape torch.Size([2, 128]) in the checkpoint and torch.Size([4, 128]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


4
4


In [8]:
kg_manager.llm_df

,source_id,target_id,split,labels,prompt_feature_title,prompt_feature_genres
0,447,2635,train,1,Parenthood (1989),"['Comedy', 'Drama']"
1,81,1486,train,1,Back to the Future Part II (1989),"['Adventure', 'Comedy', 'Sci-Fi']"
2,488,3614,train,1,Ocean's Eleven (2001),"['Crime', 'Thriller']"
3,33,1071,train,1,Mars Attacks! (1996),"['Action', 'Comedy', 'Sci-Fi']"
4,461,6342,train,1,Shut Up & Sing (2006),['Documentary']
...,...,...,...,...,...,...
64528,505,5984,val,0,Proof (2005),['Drama']
64529,445,1446,val,0,"Nightmare on Elm Street 2: Freddy's Revenge, A...",['Horror']
64530,8,9234,val,0,Who Killed Chea Vichea? (2010),['Documentary']
64531,225,1784,val,0,Police Academy 5: Assignment: Miami Beach (1988),"['Comedy', 'Crime']"


Next we generate a vanilla llm dataset and tokenize it for training.

In [9]:
dataset_vanilla = kg_manager.generate_vanilla_dataset(
    vanilla_bert_classifier.tokenize_function
)

Map:   0%|          | 0/24201 [00:00<?, ? examples/s]

Map:   0%|          | 0/20166 [00:00<?, ? examples/s]

Map:   0%|          | 0/20166 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/24201 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/20166 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/20166 [00:00<?, ? examples/s]

In [10]:
dataset_vanilla["train"][0]["prompt"]

"447[SEP]2635[SEP]Parenthood (1989)[SEP]['Comedy', 'Drama']"

In [11]:
dataset_vanilla

DatasetDict({
    train: Dataset({
        features: ['source_id', 'target_id', 'split', 'labels', 'prompt_feature_title', 'prompt_feature_genres', 'prompt', 'input_ids', 'attention_mask', 'token_type_ranges', 'token_type_ids'],
        num_rows: 24201
    })
    val: Dataset({
        features: ['source_id', 'target_id', 'split', 'labels', 'prompt_feature_title', 'prompt_feature_genres', 'prompt', 'input_ids', 'attention_mask', 'token_type_ranges', 'token_type_ids'],
        num_rows: 20166
    })
    test: Dataset({
        features: ['source_id', 'target_id', 'split', 'labels', 'prompt_feature_title', 'prompt_feature_genres', 'prompt', 'input_ids', 'attention_mask', 'token_type_ranges', 'token_type_ids'],
        num_rows: 20166
    })
})

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [12]:
vanilla_bert_classifier.train_model_on_data(
    dataset_vanilla, epochs=EPOCHS, batch_size=BATCH_SIZE_LLM
)

  0%|          | 0/760 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.6897, 'grad_norm': 1.0061800479888916, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.11}
{'loss': 0.6897, 'grad_norm': 0.8246354460716248, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.21}
{'loss': 0.6862, 'grad_norm': 1.0328469276428223, 'learning_rate': 3e-06, 'epoch': 0.32}
{'loss': 0.6875, 'grad_norm': 1.200447916984558, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.42}
{'loss': 0.6839, 'grad_norm': 0.8768653869628906, 'learning_rate': 5e-06, 'epoch': 0.53}
{'loss': 0.6814, 'grad_norm': 0.8105555772781372, 'learning_rate': 6e-06, 'epoch': 0.63}
{'loss': 0.6802, 'grad_norm': 0.7252908945083618, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.74}
{'loss': 0.6754, 'grad_norm': 0.49430352449417114, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.84}
{'loss': 0.6778, 'grad_norm': 0.39622610807418823, 'learning_rate': 9e-06, 'epoch': 0.95}


  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.7063984274864197, 'eval_accuracy': 0.5, 'eval_runtime': 15.6285, 'eval_samples_per_second': 1290.334, 'eval_steps_per_second': 5.055, 'epoch': 1.0}
{'loss': 0.6739, 'grad_norm': 0.40631821751594543, 'learning_rate': 1e-05, 'epoch': 1.05}
{'loss': 0.6724, 'grad_norm': 0.38505494594573975, 'learning_rate': 1.1000000000000001e-05, 'epoch': 1.16}
{'loss': 0.6689, 'grad_norm': 0.45800697803497314, 'learning_rate': 1.2e-05, 'epoch': 1.26}
{'loss': 0.6713, 'grad_norm': 0.25119832158088684, 'learning_rate': 1.3000000000000001e-05, 'epoch': 1.37}
{'loss': 0.6662, 'grad_norm': 0.36643391847610474, 'learning_rate': 1.4000000000000001e-05, 'epoch': 1.47}
{'loss': 0.668, 'grad_norm': 0.25897446274757385, 'learning_rate': 1.5e-05, 'epoch': 1.58}
{'loss': 0.6663, 'grad_norm': 0.3056357502937317, 'learning_rate': 1.6000000000000003e-05, 'epoch': 1.68}
{'loss': 0.6678, 'grad_norm': 0.33235061168670654, 'learning_rate': 1.7000000000000003e-05, 'epoch': 1.79}
{'loss': 0.6572, 'grad_norm':

...

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.5125658512115479, 'eval_accuracy': 0.7550828126549638, 'eval_runtime': 15.924, 'eval_samples_per_second': 1266.393, 'eval_steps_per_second': 4.961, 'epoch': 7.0}
{'loss': 0.5119, 'grad_norm': 2.4244143962860107, 'learning_rate': 1.730769230769231e-05, 'epoch': 7.05}
{'loss': 0.509, 'grad_norm': 2.1597790718078613, 'learning_rate': 1.5384615384615387e-05, 'epoch': 7.16}
{'loss': 0.5117, 'grad_norm': 2.2433955669403076, 'learning_rate': 1.3461538461538462e-05, 'epoch': 7.26}
{'loss': 0.5208, 'grad_norm': 1.3145318031311035, 'learning_rate': 1.153846153846154e-05, 'epoch': 7.37}
{'loss': 0.5018, 'grad_norm': 1.2993314266204834, 'learning_rate': 9.615384615384616e-06, 'epoch': 7.47}
{'loss': 0.5049, 'grad_norm': 1.458736538887024, 'learning_rate': 7.692307692307694e-06, 'epoch': 7.58}
{'loss': 0.5078, 'grad_norm': 1.309504747390747, 'learning_rate': 5.76923076923077e-06, 'epoch': 7.68}
{'loss': 0.5019, 'grad_norm': 1.77482271194458, 'learning_rate': 3.846153846153847e-06, '

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.5084526538848877, 'eval_accuracy': 0.7593474164435188, 'eval_runtime': 15.8522, 'eval_samples_per_second': 1272.123, 'eval_steps_per_second': 4.984, 'epoch': 8.0}
{'train_runtime': 363.468, 'train_samples_per_second': 532.669, 'train_steps_per_second': 2.091, 'train_loss': 0.5964150265643471, 'epoch': 8.0}


AttributeError: 'VanillaBertClassifier' object has no attribute 'device'

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.